In [1]:
PATH_TRAIN = "../competition_results/data/train_LPC_RP.csv" 
PATH_TEST = "../competition_results/data/test_LPC_RP.csv"
PATH_SAVE_TRAIN = "../competition_results/data/train_LPC_RP_fixed_period.csv"
PATH_SAVE_TEST = "../competition_results/data/test_LPC_RP_fixed_period.csv"

In [2]:
from utils.coosys import cartesian_to_kepler as ctk, cartesian_to_quaternion as ctq
from utils.coosys import kepler_to_cartesian as ktc, quaternion_to_cartesian as qtc
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.model_selection import train_test_split as tts
from scipy.interpolate import UnivariateSpline as US
from sklearn.pipeline import Pipeline as PL
from sklearn.preprocessing import StandardScaler as SS
from scipy import signal
import spectrum
from tqdm.auto import tqdm
tqdm.pandas()

from matplotlib import pyplot as plt
%matplotlib inline

D:\Anaconda3\envs\Standard\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
train_data = pd.read_csv(PATH_TRAIN, encoding="utf8")
train_data["timestamp"] = pd.to_datetime(train_data["epoch"]).apply(pd.Timestamp.timestamp)
train_data = train_data.sort_values(by="timestamp")

test_data = pd.read_csv(PATH_TEST, encoding="utf8")
test_data["timestamp"] = pd.to_datetime(test_data["epoch"]).apply(pd.Timestamp.timestamp)
test_data = test_data.sort_values(by="timestamp")

In [4]:
gamma_km = 398603

def transform_row(row, func):
    return func(row.values)

def transform_dataset(data, func=lambda data: ctk(data, gamma_km), 
                      columns_from=("x", "y", "z", "Vx", "Vy", "Vz"), 
                      columns_to=("a", "e", "inclination", "longitude", "argument", "anomaly")):
    new_data = data.loc[:, list(columns_from)].progress_apply(transform_row, axis=1, result_type="expand", args=(func, ))
    new_data.columns = columns_to
    return data.drop(list(columns_from), axis=1).join(new_data)

In [5]:
train_kepler = transform_dataset(train_data, func=lambda data: ctk(data, gamma_km),
                                 columns_from=("x", "y", "z", "Vx", "Vy", "Vz"),
                                 columns_to=("a", "e", "inclination", "longitude", "argument", "anomaly"))
train_kepler = transform_dataset(train_kepler, func=lambda data: ctk(data, gamma_km), 
                                 columns_from=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"),
                                 columns_to=("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"))

In [6]:
test_kepler = transform_dataset(test_data, func=lambda data: ctk(data, gamma_km), 
                                columns_from=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"),
                                columns_to=("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"))

In [7]:
def phase_regression(phases):
    periods = 0
    new_phases = phases.copy()
    for i in range(1, phases.size):
        if phases[i] < phases[i - 1]:
            periods += 1
        new_phases[i] += 2 * np.pi * periods
    return new_phases

def phase_degression(phases):
    new_phases = np.zeros_like(phases)
    for i in range(phases.size):
        new_phases[i] = phases[i] - 2 * np.pi * np.trunc(phases[i] / 2 / np.pi) 
    return new_phases

In [8]:
def t_to_features(t):
    return t.reshape(-1, 1)

def get_new_t(t, anomaly, anomaly_sim, model=LR, **args):
    anomaly = phase_regression(anomaly)
    anomaly_sim = phase_regression(anomaly_sim)
    
    anomaly_sim += anomaly[0] - anomaly_sim[0]
    
    anomaly_points = np.linspace(min(np.min(anomaly), np.min(anomaly_sim)), 
                                 max(np.max(anomaly), np.max(anomaly_sim)), 
                                 10 * (anomaly.size + anomaly_sim.size))
    t_anomaly = US(anomaly, t, k=1, s=0.)
    t_anomaly_sim = US(anomaly_sim, t, k=1, s=0.)
    
    base = model(**args).fit(t_to_features(t_anomaly(anomaly_points)), t_anomaly_sim(anomaly_points))
    
    return lambda x: base.predict(t_to_features(x))

In [9]:
def fix_sat(train, test, sat_id, model=LR, **args):
    train_sat = train[train.sat_id == sat_id]
    test_sat = test[test.sat_id == sat_id]
    
    train_anomaly_sim = train_sat["anomaly_sim"].to_numpy()
    test_anomaly_sim = test_sat["anomaly_sim"].to_numpy()
    train_t = train_sat["timestamp"].to_numpy()
    test_t = test_sat["timestamp"].to_numpy()
    
    all_anomaly_sim = np.concatenate([train_anomaly_sim, test_anomaly_sim], axis=0)
    all_t = np.concatenate([train_t, test_t], axis=0)
    
    train_anomaly_true = train_sat["anomaly"].to_numpy()
    
    new_t = get_new_t(train_t, train_anomaly_true, train_anomaly_sim, model, **args)
    
    for coordinate in ("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"):
        train_coordinate = train_sat[coordinate].to_numpy()
        test_coordinate = test_sat[coordinate].to_numpy()
        all_coordinate = np.concatenate([train_coordinate, test_coordinate])
        
        coordinate_t = US(all_t, all_coordinate, k=1, s=0.)
        new_coordinate = coordinate_t(new_t(all_t))
        
        train_sat[coordinate] = new_coordinate[:train_t.size]
        test_sat[coordinate] = new_coordinate[train_t.size:]

    return train_sat, test_sat


def fix_sat_train(train, sat_id, model=LR, **args):
    train_sat = train[train.sat_id == sat_id]
    
    train_anomaly_sim = train_sat["anomaly_sim"].to_numpy()
    train_t = train_sat["timestamp"].to_numpy()
    
    train_anomaly_true = train_sat["anomaly"].to_numpy()
    
    new_t = get_new_t(train_t, train_anomaly_true, train_anomaly_sim, model, **args)
    
    for coordinate in ("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"):
        train_coordinate = train_sat[coordinated].to_numpy()
        
        coordinate_t = US(train_t, train_coordinate, k=1, s=0.)
        new_coordinate = coordinate_t(new_t(all_t))
                       
        train_sat[coordinate] = new_coordinate[:train_t.size]
    
    return train_sat

In [10]:
sats_to_predict = set(test_data["sat_id"].unique())
sats_others = set(train_data["sat_id"]) - sats_to_predict

In [11]:
new_train = []
new_test = []

for sat_id in tqdm(sats_to_predict):
    train_sat, test_sat = fix_sat(train_kepler, test_kepler, sat_id, fit_intercept=False)
    new_train.append(train_sat)
    new_test.append(test_sat)
    
for sat_id in tqdm(sats_others):
    train_sat = fix_sat_train(train_kepler, sat_id, fit_intercept=False)
    new_train.append(train_sat)
    
new_train = pd.concat(new_train)
new_test = pd.concat(new_test)

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

...

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\envs\Standard\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [12]:
new_train = transform_dataset(new_train, func=lambda data: ktc(data, gamma_km),
                              columns_from=("a", "e", "inclination", "longitude", "argument", "anomaly"),
                              columns_to=("x", "y", "z", "Vx", "Vy", "Vz"))
new_train = transform_dataset(new_train, func=lambda data: ktc(data, gamma_km), 
                              columns_from=("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"),
                              columns_to=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"))

In [13]:
new_test = transform_dataset(new_test, func=lambda data: ktc(data, gamma_km), 
                             columns_from=("a_sim", "e_sim", "inclination_sim", "longitude_sim", "argument_sim", "anomaly_sim"),
                             columns_to=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"))

In [14]:
new_train.drop("timestamp", axis=1).to_csv(PATH_SAVE_TRAIN, index=False)
new_test.drop("timestamp", axis=1).to_csv(PATH_SAVE_TEST, index=False)